In [2]:
try:
  import cudaq
  import cudaq_solvers as solvers
  import dynamiqs as dq
except(ImportError):
  %pip install cudaq
  %pip install cudaq-solvers
  %pip install dynamiqs

  import cudaq
  import cudaq_solvers as solvers
  import dynamiqs as dq

import numpy as np
from scipy.optimize import minimize
from scipy.linalg import qr, expm
import jax.numpy as jnp
import math

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 MB 4.6 MB/s eta 0:00:00
  Created wheel for cudaq: filename=cudaq-0.12.0.post1-py3-none-any.whl size=7140 sha256=86d4203345a30dacc459221d939562abbfb174fb32579db6193c92446464d70b
  Stored in directory: /root/.cache/pip/wheels/28/46/73/06c85cbac5fac9d1aef5f8ffd2e96d1acda5bd101fdce3c0b9
Successfully built cudaq
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Make any vector into unit vector
def normalize(data):
   magnitude = np.vdot(data,data)
   return data/magnitude

Using Q-R decomposition to decompose a non-unitary $mxm$ matrix $H$ into a unitary matrix $Q$ and a upper triangular matrix $R$.

Then using this new unitary version of our hamiltonian, we create a unitary gate operation "Q_gate", and put it into a $n = \log_2(m)$ qubit circuit.

 As every circuit begins in the $|\psi_0\rangle ≡ |0_10_20_3...0_n\rangle$ state, this circuit performs the operation

$|\psi_f\rangle = Q|\psi_0\rangle$

In [1]:
def QR_circuit(H: np.array, repeat=1) -> cudaq.Kernel:

    kernel = cudaq.make_kernel()
    n = int(math.log2(len(H)))
    q = kernel.qalloc(n)
    qubits=[]
    for i in range(0,n):
      qubits.append(q[n-i-1])

    Q,R = qr(H)
    cudaq.register_operation("Q_gate",Q)

    for i in range(0,repeat):

      kernel.Q_gate(*qubits)

    return kernel


def state_out(circuit: cudaq.Kernel, norm = True):
  if norm:
    return normalize(np.array(cudaq.get_state(circuit), dtype=jnp.complex128))
  else:
    return np.array(cudaq.get_state(circuit), dtype=jnp.complex128)





NameError: name 'np' is not defined

Here, we do a similar thing, but once we Q-R decompose the non-unitary hamiltonian $H$, we propogate our initial state $|\psi_0⟩$ through time by a small increment $t$ by acting on it by the operator $e^{-iHt}$, replacing the non unitary $H$ with a unitary factor $Q$.

Again starting at the intial state $|\psi_0\rangle ≡ |0_10_20_3...0_n\rangle$

We find the ith state in the interval $0$ to $T$, by taking

$|\psi_i⟩ = e^{-iQt_i}|\psi_0⟩$

where $t_i$ is defined to be $t×i$, with $i ∈ {0,T/t}$

In [4]:
def evolve(H: np.array, T: int, t=0.001, steps=False) -> cudaq.Kernel:
    states = []
    kernel = cudaq.make_kernel()
    n = int(math.log2(len(H)))
    q = kernel.qalloc(n)
    states.append(state_out(kernel))


    repeat = T*int(1/t)

    for i in range(0,repeat):


      kernel = cudaq.make_kernel()
      n = int(math.log2(len(H)))
      q = kernel.qalloc(n)
      qubits=[]
      for j in range(0,n):
        qubits.append(q[n-j-1])

      Q,R = qr(H)


      Q_step =  expm(-1j*Q*(t*i))
      cudaq.register_operation("Q_step",Q_step)


      if steps:
        print(f"step {i}")
      kernel.Q_step(*qubits)
      states.append(state_out(kernel))



    return states



def state_out(circuit: cudaq.Kernel, norm = True):
  if norm:
    return normalize(np.array(cudaq.get_state(circuit), dtype=jnp.complex128))
  else:
    return np.array(cudaq.get_state(circuit), dtype=jnp.complex128)



In [ ]:
def circuit(n: int) -> cudaq.Kernel:

    kernel, thetas = cudaq.make_kernel(list)
    q = kernel.qalloc(n)

    for i in range(0,n):
      kernel.rx(thetas[i],q[i])
      if i > 0:
        kernel.cx(q[0],q[i])

    return kernel


def optimize(circuit, psi_exp):

  def difference(thetas):
    return 1 - np.vdot(psi_exp, state_out(circuit,thetas))

  guess=np.random.rand(len(psi_exp))


  result = minimize(difference,guess)

  return result


State out vector: Used to return state $|\psi_f\rangle$ after circuit is applied on intial state $|\psi_0\rangle$

In [5]:
def state_out(circuit: cudaq.Kernel, norm = True):
  if norm:
    return normalize(np.array(cudaq.get_state(circuit), dtype=jnp.complex128))
  else:
    return np.array(cudaq.get_state(circuit), dtype=jnp.complex128)



In [6]:
# Define parameters
g2 = 1.0
epsilon_d = -4
kappa_b = 10
T = 3  # Simulation time
na = 32# Hilbert space truncation for memory mode
nb = 4# Hilbert space truncation for buffer mode


a,b = dq.destroy(na,nb)
psi0 = dq.tensor(dq.fock(na, 0), dq.fock(nb, 0))

H_2PH = g2 * (a @ a @ b.dag() + a.dag() @ a.dag() @ b)

# Define the Driving Hamiltonian
H_D = epsilon_d * (b + b.dag())

# Define Hamiltonian (using g2 and epsilon_d)
H = H_2PH + H_D
H_num = np.array(H)

Q,R = qr(H)

Q = dq.asqarray(Q, dims=(na,nb))

steps = 300
tsave = np.linspace(0,T,steps)

res = dq.sesolve(Q, psi0, tsave)

|██████████| 100.0% ◆ elapsed 10.65ms ◆ remaining 0.00ms


In [7]:
psi1 = normalize(res.states[30])

In [8]:
states = evolve(H,3,t=0.01)


step 0
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
step 51
step 52
step 53
step 54
step 55
step 56
step 57
step 58
step 59
step 60
step 61
step 62
step 63
step 64
step 65
step 66
step 67
step 68
step 69
step 70
step 71
step 72
step 73
step 74
step 75
step 76
step 77
step 78
step 79
step 80
step 81
step 82
step 83
step 84
step 85
step 86
step 87
step 88
step 89
step 90
step 91
step 92
step 93
step 94
step 95
step 96
step 97
step 98
step 99
step 100
step 101
step 102
step 103
step 104
step 105
step 106
step 107
step 108
step 109
step 110
step 111
step 112
step 113
step 114
step 115
step 116
step 117
step 118
step 119
step 120
step 121
step 122
ste

In [1]:
i=0
min_diff = 10**10
for j in range(0, len(states)):

  diff = 1- np.vdot(states[j],psi1)

  if diff < min_diff:
    min_diff = diff
    i = j



print(i)
print(np.vdot(states[3],psi1))




NameError: name 'states' is not defined

In [60]:
states[12].shape = (128,1)

array([[ 9.84704650e-01+9.05472914e-13j],
       [ 2.49263763e-14-1.08071270e-01j],
       [ 4.75411737e-03-8.30411592e-13j],
       [ 4.07412927e-13-1.55334184e-04j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [-1.18861011e-03-7.63325371e-13j],
       [-2.76636491e-13+9.01850556e-05j],
       [-3.63029751e-06-6.58023340e-13j],
       [ 1.46497964e-14-4.96664074e-08j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 1.22343286e-06+6.17573979e-15j],
       [-1.05010826e-14+6.80779826e-08j],
       [ 9.94041680e-10-1.29790308e-15j],
       [-1.84170902e-17+1.64197545e-12j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000

In [59]:
psi1

QArray: shape=(128, 1), dims=(32, 4), dtype=complex64, layout=dense
[[ 8.9538401e-01+1.6171850e-11j]
 [ 1.2264478e-12-2.6503196e-01j]
 [ 3.1859737e-02-1.4776140e-11j]
 [ 2.0385122e-11-2.8416486e-03j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j]
 [-7.9688523e-03-1.3735437e-11j]
 [-1.3830960e-11+1.6500582e-03j]
 [-1.8156161e-04-1.3089965e-11j]
 [ 7.5760042e-13-6.7853271e-06j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 6.1284161e-05+8.4572854e-13j]
 [-5.8923531e-13+9.3053168e-06j]
 [ 3.7422376e-07-1.7758914e-13j]
 [-6.9372939e-15+1.2421114e-09j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j]
 [-3.8756431e-07-2.6848442e-14j]
 [ 5.8206287e-15+1.6957916e-08j]
 [ 3.5685652e-11-5.3309146e-16j]
 [-2.9765356e-18+3.8915633e-12j]
 [ 0.000

In [32]:
for i in range (1,len(states)):
  print(states[i]- states[i-1])

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[-1.26729471e-02+6.82880159e-13j  1.70866583e-14-9.85471076e-02j
  3.94118656e-03-6.263318

In [ ]:
def circuit2(n: int) -> cudaq.Kernel:

    kernel, thetas = cudaq.make_kernel(list)
    q = kernel.qalloc(n)


    for i in range(0,n):
      kernel.rx(thetas[i],q[i])
      if i > 0:
        kernel.cx(q[0],q[i])
    for i in range(0,n):
      kernel.rx(thetas[i],q[i])
      if i > 0:
        kernel.cx(q[0],q[i])

    return kernel

In [ ]:
np.vdot(state2,psi_exp2)

np.complex128(-5.490306765341496e-11-3.9237163232084146e-16j)

In [ ]:
print(cudaq.draw(k))


        ╭───╮   ╭─────────╮               
q0 : ───┤ h ├───┤ rx(0.4) ├──●────●────●──
     ╭──┴───┴──╮╰─────────╯╭─┴─╮  │    │  
q1 : ┤ rx(0.2) ├───────────┤ x ├──┼────┼──
     ├─────────┤           ╰───╯╭─┴─╮  │  
q2 : ┤ rx(0.1) ├────────────────┤ x ├──┼──
     ├─────────┤                ╰───╯╭─┴─╮
q3 : ┤ rx(0.2) ├─────────────────────┤ x ├
     ╰─────────╯                     ╰───╯



In [ ]:
def apply_with_initial_state(psi0, circuit_kernel, thetas, n):
    # Set global initial statevector
    cudaq.set_state(psi0)

    # Run the circuit on that state
    return cudaq.sample(circuit_kernel, thetas)

In [ ]:
n=5
psi0 = np.zeros(2**n,dtype=complex)

In [ ]:
apply_with_initial_state(psi0,k,[0.2,0.2,0.2,0.2,0.2],n)

NameError: name 'apply_with_initial_state' is not defined

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



AttributeError: module 'cudaq' has no attribute 'from_state'

During handling of the above exception, another exception occurred:

AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
AttributeError: module 'cudaq' has no attribute 'from_state'

During handling of the above exception, another exception occurred:

AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

TypeError: object of type 'NoneType' has no len()

During handling of the above exception, another exception occurred:

AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
AttributeError: module 'cudaq' has no attribute 'from_state'

During handling of the above exception, another exception occurred:

AttributeError: 'A